In [1]:
from __future__ import print_function

import keras
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, merge
from keras.layers import merge
from keras.layers import LSTM, GRU
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

C:\Users\this pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from functools import reduce
import tarfile
import numpy as np
import re

In [3]:
train_model = 1
train_epochs = 100
load_model = 0
batch_size = 32
lstm_size = 64
test_qualitative = 0
user_questions = 0

In [4]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [5]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [6]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [7]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [8]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [9]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]
print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: single_supporting_fact_10k


C:\Users\this pc\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [10]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    print(story,"\n",q,answer)
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Mary', '?'] bathroom
['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'Sandra', 'moved', 'to', 'the', 'garden', '.'] 
 ['Where', 'is', 'Daniel', '?'] hallway
['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'Sandra', 'moved', 'to', 'the', 'garden', '.', 'John', 'moved', 'to', 'the', 'office', '.', 'Sandra', 'journeyed', 'to', 'the', 'bathroom', '.'] 
 ['Where', 'is', 'Daniel', '?'] hallway
['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'Sandra', 'moved', 'to', 'the', 'garden', '.', 'John', 'moved', 'to', 'the', 'office', '.', 'Sandra', 'journeyed', 'to', 'the', 'bathroom', '.', 'Mar

 ['Where', 'is', 'John', '?'] bedroom
['Mary', 'moved', 'to', 'the', 'kitchen', '.', 'Sandra', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'to', 'the', 'garden', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'journeyed', 'to', 'the', 'office', '.', 'Mary', 'went', 'to', 'the', 'bathroom', '.', 'Sandra', 'travelled', 'to', 'the', 'garden', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Sandra', '?'] garden
['Mary', 'moved', 'to', 'the', 'kitchen', '.', 'Sandra', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'to', 'the', 'garden', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'journeyed', 'to', 'the', 'office', '.', 'Mary', 'went', 'to', 'the', 'bathroom', '.', 'Sandra', 'travelled', 'to', 'the', 'garden', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'to', 'the', 'bedroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Sandra', '?'] hallway
['John', '

 ['Where', 'is', 'Daniel', '?'] kitchen
['Sandra', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'journeyed', 'to', 'the', 'office', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'moved', 'to', 'the', 'garden', '.', 'Daniel', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'bedroom', '.', 'John', 'went', 'back', 'to', 'the', 'office', '.'] 
 ['Where', 'is', 'John', '?'] office
['Sandra', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'journeyed', 'to', 'the', 'office', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'moved', 'to', 'the', 'garden', '.', 'Daniel', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'bedroom', '.', 'John', 'went', 'back', 'to', 'the', 'office', '.', 'Mary', 'travelled', 'to', 'the', 'office', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bedroom', '.'] 
 ['

 ['Where', 'is', 'Sandra', '?'] hallway
['Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'journeyed', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'hallway', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'Mary', 'went', 'to', 'the', 'kitchen', '.', 'John', 'went', 'to', 'the', 'garden', '.'] 
 ['Where', 'is', 'Mary', '?'] kitchen
['Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'journeyed', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'hallway', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'Mary', 'went', 'to', 'the', 'kitchen', '.', 'John', 'went', 'to', 'the', 'garden', '.', 'John', 'moved', 'to', 'the', 'office', '.', 'Daniel', 'went', 'back', 'to', 'the', 'bedroom', '.'] 
 ['Where', 'is', 'John', '?'] office
['Sandra'

['Sandra', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'went', 'to', 'the', 'hallway', '.', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'journeyed', 'to', 'the', 'office', '.', 'Sandra', 'travelled', 'to', 'the', 'hallway', '.', 'Sandra', 'moved', 'to', 'the', 'bedroom', '.'] 
 ['Where', 'is', 'Mary', '?'] garden
['Sandra', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'went', 'to', 'the', 'hallway', '.', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'journeyed', 'to', 'the', 'office', '.', 'Sandra', 'travelled', 'to', 'the', 'hallway', '.', 'Sandra', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'moved', 'to', 'the', 'office', '.', 'Mary', 'journeyed', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Daniel', '?'] office
['Sandra', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'went', 'to', 'the', 'hallway', '.', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'journeyed', 'to', 'the', 'office', '.', 'Sandra', 

...

 ['Where', 'is', 'Mary', '?'] office
['Mary', 'moved', 'to', 'the', 'hallway', '.', 'John', 'went', 'back', 'to', 'the', 'bedroom', '.', 'Mary', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Mary', 'travelled', 'to', 'the', 'office', '.', 'Daniel', 'went', 'back', 'to', 'the', 'office', '.', 'John', 'moved', 'to', 'the', 'hallway', '.', 'John', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Mary', 'travelled', 'to', 'the', 'garden', '.'] 
 ['Where', 'is', 'John', '?'] bathroom
['Mary', 'moved', 'to', 'the', 'hallway', '.', 'John', 'went', 'back', 'to', 'the', 'bedroom', '.', 'Mary', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Mary', 'travelled', 'to', 'the', 'office', '.', 'Daniel', 'went', 'back', 'to', 'the', 'office', '.', 'John', 'moved', 'to', 'the', 'hallway', '.', 'John', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Mary', 'travelled', 'to', 'the', 'garden', '.', 'Daniel', 'went', 'back', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'kitchen', '.'] 
 ['Where'

['Mary', 'went', 'to', 'the', 'bathroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'office', '.', 'John', 'journeyed', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'back', 'to', 'the', 'garden', '.'] 
 ['Where', 'is', 'Sandra', '?'] office
['Mary', 'went', 'to', 'the', 'bathroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'office', '.', 'John', 'journeyed', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'back', 'to', 'the', 'garden', '.', 'Sandra', 'journeyed', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Daniel', '?'] garden
['Mary', 'went', 'to', 'the', 'bathroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Sandra', 'moved', 'to', 'the', 'office', '.', 'John', 'journeyed', 'to', 'the', 'hallway', '.', 'Daniel', 'went',

['Sandra', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'journeyed', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'John', 'went', 'to', 'the', 'office', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bathroom', '.'] 
 ['Where', 'is', 'Daniel', '?'] garden
['Sandra', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'journeyed', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'John', 'went', 'to', 'the', 'office', '.', 'Sandra', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Sandra', 'travelled', 'to', 'the', 'office', '.'] 
 ['Where', 'is', 'Sandra', '?'] office
['Sandra', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'journeyed', 'to', 'the', 'garden', '.', 'Mary', 'moved', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'hallway', '.', 'John', 'we

['Sandra', 'went', 'to', 'the', 'bedroom', '.', 'John', 'travelled', 'to', 'the', 'office', '.', 'John', 'travelled', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Sandra', 'went', 'back', 'to', 'the', 'garden', '.', 'John', 'went', 'back', 'to', 'the', 'office', '.'] 
 ['Where', 'is', 'Sandra', '?'] garden
['Sandra', 'went', 'to', 'the', 'bedroom', '.', 'John', 'travelled', 'to', 'the', 'office', '.', 'John', 'travelled', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Sandra', 'went', 'back', 'to', 'the', 'garden', '.', 'John', 'went', 'back', 'to', 'the', 'office', '.', 'Daniel', 'went', 'to', 'the', 'hallway', '.', 'Daniel', 'went', 'to', 'the', 'bathroom', '.'] 
 ['Where', 'is', 'Sandra', '?'] garden
['Sandra', 'went', 'to', 'the', 'bedroom', '.', 'John', 'travelled', 'to', 'the', 'office', '.', 'John', 'travelled', 'to', 'the', 'bedroom', '.', 'Sandra', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Sandra

 ['Where', 'is', 'Daniel', '?'] kitchen
['Sandra', 'moved', 'to', 'the', 'bathroom', '.', 'Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'kitchen', '.', 'Daniel', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'travelled', 'to', 'the', 'kitchen', '.', 'Sandra', 'went', 'to', 'the', 'bathroom', '.'] 
 ['Where', 'is', 'Sandra', '?'] bathroom
['Sandra', 'moved', 'to', 'the', 'bathroom', '.', 'Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'kitchen', '.', 'Daniel', 'moved', 'to', 'the', 'kitchen', '.', 'Mary', 'travelled', 'to', 'the', 'kitchen', '.', 'Sandra', 'went', 'to', 'the', 'bathroom', '.', 'John', 'went', 'back', 'to', 'the', 'bathroom', '.', 'Daniel', 'moved', 'to', 'the', 'hallway', '.'] 
 ['Where', 'is', 'Daniel', '?'] hallway
['Sandra', 'moved', 'to', 'the', 'bathroom', '.', 'Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'kitchen', '.', 'Daniel', 'moved',

In [11]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...


In [12]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx_word = dict((i+1, c) for i,c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 22)
answers_test shape: (1000, 22)
-
Compiling...


In [13]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

print('Input sequence:', input_sequence)
print('Question:', question)

Input sequence: Tensor("input_1:0", shape=(?, 68), dtype=float32)
Question: Tensor("input_2:0", shape=(?, 4), dtype=float32)


In [14]:
# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
print('Input encoded m', input_encoded_m)
input_encoded_c = input_encoder_c(input_sequence)
print('Input encoded c', input_encoded_c)
question_encoded = question_encoder(question)
print('Question encoded', question_encoded)

Input encoded m Tensor("sequential_1/dropout_1/cond/Merge:0", shape=(?, 68, 64), dtype=float32)
Input encoded c Tensor("sequential_2/dropout_2/cond/Merge:0", shape=(?, 68, 4), dtype=float32)
Question encoded Tensor("sequential_3/dropout_3/cond/Merge:0", shape=(?, 4, 64), dtype=float32)


In [15]:
# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)
match = merge([input_encoded_m, question_encoded], mode='dot', dot_axes=(2, 2))
match = Activation('softmax')(match)
print('Match shape', match)

Match shape Tensor("activation_1/truediv:0", shape=(?, 68, 4), dtype=float32)


C:\Users\this pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.
C:\Users\this pc\Anaconda3\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [16]:
# add the match matrix with the second input vector sequence
response = merge([match, input_encoded_c], mode='sum')  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)
print('Response shape', response)

# concatenate the response vector with the question vector sequence
answer = merge([response, question_encoded], mode='concat')
print('Answer shape', answer)

Response shape Tensor("permute_1/transpose:0", shape=(?, 4, 68), dtype=float32)
Answer shape Tensor("merge_3/concat:0", shape=(?, 4, 132), dtype=float32)


C:\Users\this pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\this pc\Anaconda3\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\this pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  import sys


In [17]:
# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
#answer = LSTM(lstm_size, return_sequences=True)(answer)  # Generate tensors of shape 32
#answer = Dropout(0.3)(answer)
answer = LSTM(lstm_size)(answer)  # Generate tensors of shape 32
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [18]:
# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
load_model = 1
if load_model == 1:
    model = keras.models.load_model('model.h5')

train_model = 0
if train_model == 1:
    # train, batch_size = 32 and epochs = 120
    model.fit([inputs_train, queries_train], answers_train, batch_size, train_epochs,
          validation_data=([inputs_test, queries_test], answers_test))
    model.save('model.h5')


C:\Users\this pc\Anaconda3\lib\site-packages\keras\engine\topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [20]:
test_qualitative = 1
if test_qualitative == 1:
    print('-------------------------------------------------------------------------------------------')
    print('Qualitative Test Result Analysis')
    for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])

-------------------------------------------------------------------------------------------
Qualitative Test Result Analysis
John travelled to the hallway . Mary journeyed to the bathroom . Where is John ? | Prediction: hallway | Ground Truth: hallway
John travelled to the hallway . Mary journeyed to the bathroom . Daniel went back to the bathroom . John moved to the bedroom . Where is Mary ? | Prediction: bathroom | Ground Truth: bathroom
John travelled to the hallway . Mary journeyed to the bathroom . Daniel went back to the bathroom . John moved to the bedroom . John went to the hallway . Sandra journeyed to the kitchen . Where is Sandra ? | Prediction: kitchen | Ground Truth: kitchen
John travelled to the hallway . Mary journeyed to the bathroom . Daniel went back to the bathroom . John moved to the bedroom . John went to the hallway . Sandra journeyed to the kitchen . Sandra travelled to the hallway . John went to the garden . Where is Sandra ? | Prediction: hallway | Ground Truth

In [26]:
user_questions =1
if user_questions == 1:
    print('-------------------------------------------------------------------------------------------')
    print('Custom User Queries (Make sure there are spaces before each word)')
    while 1:
        print('-------------------------------------------------------------------------------------------')
        print('Please input a story')
        user_story_inp = input().split(' ')
        print('Please input a query')
        user_query_inp = input().split(' ')
        user_story, user_query, user_ans = vectorize_stories([[user_story_inp, user_query_inp, '.']], word_idx, story_maxlen, query_maxlen)
        user_prediction = model.predict([user_story, user_query])
        user_prediction = idx_word[np.argmax(user_prediction)]
        print('Result')
        print(' '.join(user_story_inp), ' '.join(user_query_inp), '| Prediction:', user_prediction)

-------------------------------------------------------------------------------------------
Custom User Queries (Make sure there are spaces before each word)
-------------------------------------------------------------------------------------------
Please input a story
Murali has a lot of work. Murali is listening to music. What is Murali doing?
Please input a query
What is Murali doing?


KeyError: 'Murali'

In [24]:
word_idx

{'.': 1,
 '?': 2,
 'Daniel': 3,
 'John': 4,
 'Mary': 5,
 'Sandra': 6,
 'Where': 7,
 'back': 8,
 'bathroom': 9,
 'bedroom': 10,
 'garden': 11,
 'hallway': 12,
 'is': 13,
 'journeyed': 14,
 'kitchen': 15,
 'moved': 16,
 'office': 17,
 'the': 18,
 'to': 19,
 'travelled': 20,
 'went': 21}

In [25]:
idx_word

{1: '.',
 2: '?',
 3: 'Daniel',
 4: 'John',
 5: 'Mary',
 6: 'Sandra',
 7: 'Where',
 8: 'back',
 9: 'bathroom',
 10: 'bedroom',
 11: 'garden',
 12: 'hallway',
 13: 'is',
 14: 'journeyed',
 15: 'kitchen',
 16: 'moved',
 17: 'office',
 18: 'the',
 19: 'to',
 20: 'travelled',
 21: 'went'}